In [1]:
import os
import pandas as pd
import shutil

In [2]:
year = 2019

In [3]:
norm_ranges = list(range(2, 31))
PSO_ranges = list(range(5, 75, 5))

In [4]:
norm_ranges

[2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30]

In [5]:
len(norm_ranges)

29

In [6]:
PSO_ranges

[5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70]

In [7]:
len(PSO_ranges)

14

In [8]:
sub_folders = ['norm_'+str(n)+'_PSO_'+str(p) for n in norm_ranges for p in PSO_ranges]

In [9]:
len(sub_folders)

406

In [10]:
new_directory = 'code/2_code_for_PSO_calibration/creating_PSO_code_by_year/'+str(year) + '/sub_code'

In [11]:
sbatch_commands = []
py_codes = []
sbatches = []
for t in sub_folders:
    path = os.path.join(new_directory, t)
    if os.path.exists(path):
        shutil.rmtree(path)
    os.mkdir(path)
    sub_path = os.path.join(new_directory, t, 'sub')
    if os.path.exists(sub_path):
        shutil.rmtree(sub_path)
    os.mkdir(sub_path)
    source_py = "code/2_code_for_PSO_calibration/creating_PSO_code_by_year/4_PSO_2018_NYC_norm_2_PSO_30.py"
    dest_py = path + '/4_PSO_' + str(year) + '_NYC_' + t + '.py'
    shutil.copy(source_py, dest_py)
    
    source_sbatch = "code/2_code_for_PSO_calibration/creating_PSO_code_by_year/4_PSO_2018_NYC_norm_2_PSO_30.sbatch"
    dest_sbatch = path + '/4_PSO_' + str(year) + '_NYC_' + t + '.sbatch'
    sbatch_commands.append('sbatch norm_PSO_combination_' + str(year) + '/'+ t + '/4_PSO_' + str(year) + '_NYC_' + t + '.sbatch\n')
    shutil.copy(source_sbatch, dest_sbatch)
    py_codes.append(dest_py)
    sbatches.append(dest_sbatch)
print('finished!')

finished!


In [12]:
with open('code/2_code_for_PSO_calibration/creating_PSO_code_by_year/'+str(year) + '/sbatches_commands_'+ str(
    year) + '.txt', 'w+') as commands_file:
    for c in sbatch_commands:
        commands_file.write(c)

In [13]:
def revise_py_code(py_path):
    norm_here = py_path.split('/')[-2].split('_')[1]
    PSO_range_here = py_path.split('/')[-2].split('_')[-1]

    with open(py_path, "rt") as py_file:
        text_here = py_file.readlines()

    year_index = text_here.index('year = 2018\n')
    norm_index = text_here.index('norm = 2\n')
    PSO_range_index = text_here.index('PSO_range = 30\n')
    write_head_index = text_here.index("with open('sub/PSO_6params_NYC_norm_' + str(norm) + 'PSO_' + str(PSO_range) + '_sub_' +str(rank) + '_' + str(year) + '.csv', 'w+') as myfile:\n")
    write_row_index = text_here.index("    with open('sub/PSO_6params_NYC_norm_' + str(norm) + 'PSO_' + str(PSO_range) + '_sub_' +str(rank) + '_' + str(year) + '.csv', 'a') as myfile:\n")
    
    with open(py_path, 'w+') as updated_file:
        for indx, n in enumerate(text_here):
            if indx == year_index:
                updated_file.write('year = ' + str(year) + '\n')
            elif indx == norm_index:
                updated_file.write('norm = ' + str(norm_here) + '\n')
            elif indx == PSO_range_index:
                updated_file.write('PSO_range = ' + str(PSO_range_here) + '\n')
            elif indx == write_head_index:
                updated_file.write(
                    "with open('norm_PSO_combination_' + str(year) + '/norm_' + str(norm) +'_PSO_'+str(PSO_range) + '/sub/PSO_6params_NYC_norm_' + str(norm) + 'PSO_' + str(PSO_range) + '_sub_' +str(rank) + '_' + str(year) + '.csv', 'w+') as myfile:\n")
            elif indx == write_row_index:
                updated_file.write("    with open('norm_PSO_combination_' + str(year) + '/norm_' + str(norm) +'_PSO_'+str(PSO_range) + '/sub/PSO_6params_NYC_norm_' + str(norm) + 'PSO_' + str(PSO_range) + '_sub_' +str(rank) + '_' + str(year) + '.csv', 'a') as myfile:\n")
            else:
                updated_file.write(n)

for p in py_codes:
    revise_py_code(p)

In [14]:
def revise_sbatch_code(sbatch_path):
    norm_here = sbatch_path.split('/')[-2].split('_')[1]
    PSO_range_here = sbatch_path.split('/')[-2].split('_')[-1]
    with open(sbatch_path, "rt") as sbatch_file:
        text_here = sbatch_file.readlines()
    
    with open(sbatch_path, 'w+') as updated_file:
        for t in text_here:
            if '#SBATCH --job-name=' in t:
                updated_file.write('#SBATCH --job-name=norm_'+str(norm_here)+'_PSO_'+str(PSO_range_here)+'_' + str(year) + '\n')
            elif '#SBATCH --output=' in t:
                updated_file.write('#SBATCH --output=' + 'norm_PSO_combination_' + str(
                    year) + '/norm_' + str(norm_here) +'_PSO_'+str(PSO_range_here) + '/4_PSO_' + str(year) + '_NYC_norm_'+str(
                    norm_here)+'_PSO_'+str(PSO_range_here)+'.out\n')
            elif '#SBATCH --error=' in t:
                updated_file.write('#SBATCH --error=' + 'norm_PSO_combination_' + str(
                    year) + '/norm_' + str(norm_here) +'_PSO_'+str(PSO_range_here) + '/4_PSO_' + str(year) + '_NYC_norm_'+str(
                    norm_here)+'_PSO_'+str(PSO_range_here)+'.err\n')
            elif 'mpirun -n 40 python3' in t:
                abs_file_path = 'norm_PSO_combination_' + str(
                    year) + '/norm_' + str(norm_here) +'_PSO_'+str(PSO_range_here) + '/4_PSO_' + str(
                    year) + '_NYC_norm_'+str(norm_here)+'_PSO_'+str(PSO_range_here)+'.py'
                updated_file.write('mpirun -n 40 python3 ' + abs_file_path)
            elif 'mail' in t:
                pass
            else:
                updated_file.write(t)

for s in sbatches:
    revise_sbatch_code(s)